In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [2]:
pgi_data=pd.read_csv('/kaggle/input/doctors-suggested-fianlto-be-predicted/Doctors_Suggested_Final(To_be_predicted).csv',low_memory=False,index_col=0)
pgi_data

,Sample Inf.,Order Type,Measurement Mode,Discrete,Judgment,Positive(Diff.),Positive(Morph.),Positive(Count),WBC Abnormal,WBC Suspect,...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
0,1,0,2,4,NaN,0,0,0,0.0,0.0,...,95.6,357.0,580.0,657.0,772.0,602.0,706.0,368.0,837.0,743.0
1,1,0,2,3,NaN,0,0,0,0.0,0.0,...,104.3,332.0,725.0,759.0,725.0,940.0,741.0,366.0,913.0,786.0
2,1,0,2,3,NaN,0,0,0,0.0,0.0,...,99.0,349.0,708.0,727.0,735.0,756.0,697.0,316.0,762.0,767.0
3,1,0,2,4,NaN,0,0,0,0.0,0.0,...,100.1,365.0,667.0,712.0,754.0,708.0,653.0,368.0,753.0,709.0
4,1,0,2,3,NaN,0,0,0,0.0,0.0,...,100.6,318.0,622.0,795.0,771.0,718.0,738.0,374.0,796.0,686.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82149,1,0,2,1,1.0,0,1,0,0.0,0.0,...,60.8,297.0,566.0,679.0,446.0,884.0,569.0,248.0,637.0,641.0
82150,1,0,2,1,1.0,0,1,1,1.0,0.0,...,64.6,330.0,617.0,736.0,635.0,1163.0,603.0,238.0,669.0,573.0
82151,1,0,2,1,1.0,1,0,1,1.0,0.0,...,72.1,395.0,711.0,636.0,594.0,631.0,450.0,183.0,60.0,291.0
82152,1,0,2,1,1.0,0,1,1,1.0,1.0,...,63.5,319.0,740.0,734.0,1111.0,1591.0,720.0,271.0,651.0,552.0


In [3]:
#splitting the data 
Training=pgi_data[pgi_data['Judgment'].notna()]
Prediction=pgi_data[pgi_data['Judgment'].isna()]

**How many null values in my Data**

In [4]:
null_counts = Training.isnull().sum()

# Filter columns with null values
columns_with_nulls = null_counts[null_counts > 0]

# Display columns with the count of null values
print(columns_with_nulls)

cols_names_with_nulls = Training.columns[Training.isnull().any()].tolist()


NEUT#(10^3/uL)       501
LYMPH#(10^3/uL)      501
MONO#(10^3/uL)       501
EO#(10^3/uL)         501
BASO#(10^3/uL)       501
NEUT%(%)             501
LYMPH%(%)            501
MONO%(%)             501
EO%(%)               501
BASO%(%)             501
IG#(10^3/uL)         501
IG%(%)               501
[WBC-D(10^3/uL)]     501
[TNC-D(10^3/uL)]     501
[NEUT#&(10^3/uL)]    501
[NEUT%&(%)]          501
[LYMP#&(10^3/uL)]    501
[LYMP%&(%)]          501
[HFLC#(10^3/uL)]     501
[HFLC%(%)]           501
[BA-D#(10^3/uL)]     501
[BA-D%(%)]           501
[NE-SSC(ch)]         501
[NE-SFL(ch)]         501
[NE-FSC(ch)]         501
[LY-X(ch)]           501
[LY-Y(ch)]           501
[LY-Z(ch)]           501
[MO-X(ch)]           501
[MO-Y(ch)]           501
[MO-Z(ch)]           501
[NE-WX]              501
[NE-WY]              501
[NE-WZ]              501
[LY-WX]              501
[LY-WY]              501
[LY-WZ]              501
[MO-WX]              501
[MO-WY]              501
[MO-WZ]              501


**Our Aim is to predict the Label "Judgment"**
* Let's Drop These columns from the Training

In [5]:
Training=Training.drop(cols_names_with_nulls,axis=1)

In [6]:
# Training.isna().sum()

**No null values in the training data**

In [7]:
Prediction.isna().sum()

Sample Inf.           0
Order Type            0
Measurement Mode      0
Discrete              0
Judgment            744
                   ... 
[LY-WY]               0
[LY-WZ]               0
[MO-WX]               0
[MO-WY]               0
[MO-WZ]               0
Length: 79, dtype: int64

In [8]:
print(Training.columns.tolist())

['Sample Inf.', 'Order Type', 'Measurement Mode', 'Discrete', 'Judgment', 'Positive(Diff.)', 'Positive(Morph.)', 'Positive(Count)', 'WBC Abnormal', 'WBC Suspect', 'RBC Abnormal', 'PLT Abnormal', 'PLT Suspect', 'PLT Info.', 'Output', 'WBC(10^3/uL)', 'RBC(10^6/uL)', 'HGB(g/dL)', 'HCT(%)', 'MCV(fL)', 'MCH(pg)', 'MCHC(g/dL)', 'PLT(10^3/uL)', 'RDW-SD(fL)', 'RDW-CV(%)', 'PDW(fL)', 'MPV(fL)', 'P-LCR(%)', 'PCT(%)', 'NRBC#(10^3/uL)', 'NRBC%(%)', '[PLT-I(10^3/uL)]', 'MicroR(%)', 'MacroR(%)', '[TNC(10^3/uL)]', '[WBC-N(10^3/uL)]', '[TNC-N(10^3/uL)]', '[BA-N#(10^3/uL)]', '[BA-N%(%)]']


Let's Use the Training data to build model and test it 

**Data Modelling for Model Building**

In [9]:
X=Training.drop('Judgment',axis=1)
Y=Training['Judgment']

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=7)

**Model Defination**

In [11]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier(n_estimators=100,random_state=7)
RF.fit(x_train,y_train)

RandomForestClassifier(random_state=7)

In [12]:
y_pred=RF.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

In [14]:
print("The Accuracy is =",accuracy)
print("The Precison is ",precision)
print("The Recall is ",recall)
print("The F1 score is",f1)
print("The ROC_AUC_Score =",roc_auc)

The Accuracy is = 1.0
The Precison is  1.0
The Recall is  1.0
The F1 score is 1.0
The ROC_AUC_Score = 1.0


**Lets make validation with the prediction Dataset .I.e the Rows with Missing Judgment values .**

In [15]:
# cols_names_with_nulls = Prediction.columns[Prediction.isnull().any()].tolist()

Since we Dropped The nulls in training .Lets drop that columns here as well

In [16]:
Prediction=Prediction.drop(cols_names_with_nulls,axis=1)

In [17]:
print(cols_names_with_nulls)

['NEUT#(10^3/uL)', 'LYMPH#(10^3/uL)', 'MONO#(10^3/uL)', 'EO#(10^3/uL)', 'BASO#(10^3/uL)', 'NEUT%(%)', 'LYMPH%(%)', 'MONO%(%)', 'EO%(%)', 'BASO%(%)', 'IG#(10^3/uL)', 'IG%(%)', '[WBC-D(10^3/uL)]', '[TNC-D(10^3/uL)]', '[NEUT#&(10^3/uL)]', '[NEUT%&(%)]', '[LYMP#&(10^3/uL)]', '[LYMP%&(%)]', '[HFLC#(10^3/uL)]', '[HFLC%(%)]', '[BA-D#(10^3/uL)]', '[BA-D%(%)]', '[NE-SSC(ch)]', '[NE-SFL(ch)]', '[NE-FSC(ch)]', '[LY-X(ch)]', '[LY-Y(ch)]', '[LY-Z(ch)]', '[MO-X(ch)]', '[MO-Y(ch)]', '[MO-Z(ch)]', '[NE-WX]', '[NE-WY]', '[NE-WZ]', '[LY-WX]', '[LY-WY]', '[LY-WZ]', '[MO-WX]', '[MO-WY]', '[MO-WZ]']


In [18]:
Prediction.isna().sum()

Sample Inf.           0
Order Type            0
Measurement Mode      0
Discrete              0
Judgment            744
Positive(Diff.)       0
Positive(Morph.)      0
Positive(Count)       0
WBC Abnormal          0
WBC Suspect           0
RBC Abnormal          0
PLT Abnormal          0
PLT Suspect           0
PLT Info.             0
Output                0
WBC(10^3/uL)          0
RBC(10^6/uL)          0
HGB(g/dL)             0
HCT(%)                0
MCV(fL)               0
MCH(pg)               0
MCHC(g/dL)            0
PLT(10^3/uL)          0
RDW-SD(fL)            0
RDW-CV(%)             0
PDW(fL)               0
MPV(fL)               0
P-LCR(%)              0
PCT(%)                0
NRBC#(10^3/uL)        0
NRBC%(%)              0
[PLT-I(10^3/uL)]      0
MicroR(%)             0
MacroR(%)             0
[TNC(10^3/uL)]        0
[WBC-N(10^3/uL)]      0
[TNC-N(10^3/uL)]      0
[BA-N#(10^3/uL)]      0
[BA-N%(%)]            0
dtype: int64

In [19]:
x_valid=Prediction.drop('Judgment',axis=1)
y_valid=Prediction['Judgment']

In [20]:
y_valid_pred=RF.predict(x_valid)

In [21]:
pgi_data.loc[pgi_data['Judgment'].isna(), 'Judgment'] = y_valid_pred

In [22]:
# y_valid_pred

In [23]:
pgi_data

,Sample Inf.,Order Type,Measurement Mode,Discrete,Judgment,Positive(Diff.),Positive(Morph.),Positive(Count),WBC Abnormal,WBC Suspect,...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
0,1,0,2,4,0.0,0,0,0,0.0,0.0,...,95.6,357.0,580.0,657.0,772.0,602.0,706.0,368.0,837.0,743.0
1,1,0,2,3,0.0,0,0,0,0.0,0.0,...,104.3,332.0,725.0,759.0,725.0,940.0,741.0,366.0,913.0,786.0
2,1,0,2,3,0.0,0,0,0,0.0,0.0,...,99.0,349.0,708.0,727.0,735.0,756.0,697.0,316.0,762.0,767.0
3,1,0,2,4,0.0,0,0,0,0.0,0.0,...,100.1,365.0,667.0,712.0,754.0,708.0,653.0,368.0,753.0,709.0
4,1,0,2,3,0.0,0,0,0,0.0,0.0,...,100.6,318.0,622.0,795.0,771.0,718.0,738.0,374.0,796.0,686.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82149,1,0,2,1,1.0,0,1,0,0.0,0.0,...,60.8,297.0,566.0,679.0,446.0,884.0,569.0,248.0,637.0,641.0
82150,1,0,2,1,1.0,0,1,1,1.0,0.0,...,64.6,330.0,617.0,736.0,635.0,1163.0,603.0,238.0,669.0,573.0
82151,1,0,2,1,1.0,1,0,1,1.0,0.0,...,72.1,395.0,711.0,636.0,594.0,631.0,450.0,183.0,60.0,291.0
82152,1,0,2,1,1.0,0,1,1,1.0,1.0,...,63.5,319.0,740.0,734.0,1111.0,1591.0,720.0,271.0,651.0,552.0


In [24]:
pgi_data.isna().sum()

Sample Inf.           0
Order Type            0
Measurement Mode      0
Discrete              0
Judgment              0
                   ... 
[LY-WY]             501
[LY-WZ]             501
[MO-WX]             501
[MO-WY]             501
[MO-WZ]             501
Length: 79, dtype: int64

So The Judgment is a Categorical Column and we filled with the accurate predictions made by our Random Forest with 100 Trees

# Predicting Continous missing val cols using Linear Regression

In [25]:
pgi_data.to_csv('pgi_data_doctor_suggested_predictions_1.csv')

In [26]:
pgi_data=pd.read_csv('/kaggle/working/pgi_data_doctor_suggested_predictions_1.csv',low_memory=False,index_col=0)
pgi_data

,Sample Inf.,Order Type,Measurement Mode,Discrete,Judgment,Positive(Diff.),Positive(Morph.),Positive(Count),WBC Abnormal,WBC Suspect,...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
0,1,0,2,4,0.0,0,0,0,0.0,0.0,...,95.6,357.0,580.0,657.0,772.0,602.0,706.0,368.0,837.0,743.0
1,1,0,2,3,0.0,0,0,0,0.0,0.0,...,104.3,332.0,725.0,759.0,725.0,940.0,741.0,366.0,913.0,786.0
2,1,0,2,3,0.0,0,0,0,0.0,0.0,...,99.0,349.0,708.0,727.0,735.0,756.0,697.0,316.0,762.0,767.0
3,1,0,2,4,0.0,0,0,0,0.0,0.0,...,100.1,365.0,667.0,712.0,754.0,708.0,653.0,368.0,753.0,709.0
4,1,0,2,3,0.0,0,0,0,0.0,0.0,...,100.6,318.0,622.0,795.0,771.0,718.0,738.0,374.0,796.0,686.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82149,1,0,2,1,1.0,0,1,0,0.0,0.0,...,60.8,297.0,566.0,679.0,446.0,884.0,569.0,248.0,637.0,641.0
82150,1,0,2,1,1.0,0,1,1,1.0,0.0,...,64.6,330.0,617.0,736.0,635.0,1163.0,603.0,238.0,669.0,573.0
82151,1,0,2,1,1.0,1,0,1,1.0,0.0,...,72.1,395.0,711.0,636.0,594.0,631.0,450.0,183.0,60.0,291.0
82152,1,0,2,1,1.0,0,1,1,1.0,1.0,...,63.5,319.0,740.0,734.0,1111.0,1591.0,720.0,271.0,651.0,552.0


In [27]:
x=pgi_data.isna().sum()
lft=x.tolist()
ls=[]
for i in range(len(lft)):
    if(lft[i]>0):
        ls.append(lft[i])

In [28]:
print(ls)

[501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501]


Now Let's Impute the Continous missing value cols

In [29]:

cols_with_nulls_lst=cols_names_with_nulls
print(cols_with_nulls_lst)

['NEUT#(10^3/uL)', 'LYMPH#(10^3/uL)', 'MONO#(10^3/uL)', 'EO#(10^3/uL)', 'BASO#(10^3/uL)', 'NEUT%(%)', 'LYMPH%(%)', 'MONO%(%)', 'EO%(%)', 'BASO%(%)', 'IG#(10^3/uL)', 'IG%(%)', '[WBC-D(10^3/uL)]', '[TNC-D(10^3/uL)]', '[NEUT#&(10^3/uL)]', '[NEUT%&(%)]', '[LYMP#&(10^3/uL)]', '[LYMP%&(%)]', '[HFLC#(10^3/uL)]', '[HFLC%(%)]', '[BA-D#(10^3/uL)]', '[BA-D%(%)]', '[NE-SSC(ch)]', '[NE-SFL(ch)]', '[NE-FSC(ch)]', '[LY-X(ch)]', '[LY-Y(ch)]', '[LY-Z(ch)]', '[MO-X(ch)]', '[MO-Y(ch)]', '[MO-Z(ch)]', '[NE-WX]', '[NE-WY]', '[NE-WZ]', '[LY-WX]', '[LY-WY]', '[LY-WZ]', '[MO-WX]', '[MO-WY]', '[MO-WZ]']


In [30]:
datanull=pgi_data[pgi_data.columns[pgi_data.isnull().mean()>0]]
dataNotnull=pgi_data[pgi_data.columns[pgi_data.isnull().mean()==0]]
print(datanull)
print(dataNotnull)

       NEUT#(10^3/uL)  LYMPH#(10^3/uL)  MONO#(10^3/uL)  EO#(10^3/uL)  \
0                1.21             1.02            0.36          0.28   
1                8.30             4.03            2.00          2.28   
2                3.00             2.04            0.78          0.73   
3                3.04             2.06            0.82          0.79   
4                1.14             1.03            0.32          0.24   
...               ...              ...             ...           ...   
82149            2.83             1.67            0.93          0.21   
82150            9.68             1.13            0.96          0.01   
82151            3.53             0.17            0.03          0.00   
82152            5.22             1.75            1.07          0.00   
82153            7.59             1.25            0.66          0.00   

       BASO#(10^3/uL)  NEUT%(%)  LYMPH%(%)  MONO%(%)  EO%(%)  BASO%(%)  ...  \
0                0.15      40.0       33.8      11.9    

**Splitting the training and testing Datasets**
* Training is done with Not Null data
* Predictions are made on Null Data and filled into the original Dataset Finally (pgi_data)

In [31]:
#we are eliminating the cols with null and training the model with other features
x_train=dataNotnull[~pgi_data[cols_with_nulls_lst].isnull().any(axis=1)]
x_test=dataNotnull[pgi_data[cols_with_nulls_lst].isnull().any(axis=1)]

In [32]:
#Here y_trian dont have any null values
y_train=datanull[~datanull[cols_with_nulls_lst].isnull().any(axis=1)]
y_test=datanull[datanull[cols_with_nulls_lst].isnull().any(axis=1)]

In [33]:
y_train.head()

,NEUT#(10^3/uL),LYMPH#(10^3/uL),MONO#(10^3/uL),EO#(10^3/uL),BASO#(10^3/uL),NEUT%(%),LYMPH%(%),MONO%(%),EO%(%),BASO%(%),...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
0,1.21,1.02,0.36,0.28,0.15,40.0,33.8,11.9,9.3,5.0,...,95.6,357.0,580.0,657.0,772.0,602.0,706.0,368.0,837.0,743.0
1,8.30,4.03,2.00,2.28,0.81,47.7,23.1,11.5,13.1,4.6,...,104.3,332.0,725.0,759.0,725.0,940.0,741.0,366.0,913.0,786.0
2,3.00,2.04,0.78,0.73,0.32,43.6,29.7,11.4,10.6,4.7,...,99.0,349.0,708.0,727.0,735.0,756.0,697.0,316.0,762.0,767.0
3,3.04,2.06,0.82,0.79,0.33,43.2,29.3,11.6,11.2,4.7,...,100.1,365.0,667.0,712.0,754.0,708.0,653.0,368.0,753.0,709.0
4,1.14,1.03,0.32,0.24,0.13,39.9,36.0,11.2,8.4,4.5,...,100.6,318.0,622.0,795.0,771.0,718.0,738.0,374.0,796.0,686.0


In [34]:
#so we want to predict these cols 
y_test.head()

,NEUT#(10^3/uL),LYMPH#(10^3/uL),MONO#(10^3/uL),EO#(10^3/uL),BASO#(10^3/uL),NEUT%(%),LYMPH%(%),MONO%(%),EO%(%),BASO%(%),...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
12583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
x_train.shape

(79240, 39)

# Buidling Regression Model

In [36]:
from sklearn.linear_model import LinearRegression

model=LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [37]:
y_pred=model.predict(x_test)
#now lets find the nulls and fill that with predictions


In [38]:
y_pred = pd.DataFrame(y_pred, columns=cols_with_nulls_lst)

In [39]:
y_pred.index = y_test.index

In [40]:
cols_with_nulls_lst = [''.join(col) if isinstance(col, tuple) else col for col in cols_with_nulls_lst]
print(cols_with_nulls_lst)

['NEUT#(10^3/uL)', 'LYMPH#(10^3/uL)', 'MONO#(10^3/uL)', 'EO#(10^3/uL)', 'BASO#(10^3/uL)', 'NEUT%(%)', 'LYMPH%(%)', 'MONO%(%)', 'EO%(%)', 'BASO%(%)', 'IG#(10^3/uL)', 'IG%(%)', '[WBC-D(10^3/uL)]', '[TNC-D(10^3/uL)]', '[NEUT#&(10^3/uL)]', '[NEUT%&(%)]', '[LYMP#&(10^3/uL)]', '[LYMP%&(%)]', '[HFLC#(10^3/uL)]', '[HFLC%(%)]', '[BA-D#(10^3/uL)]', '[BA-D%(%)]', '[NE-SSC(ch)]', '[NE-SFL(ch)]', '[NE-FSC(ch)]', '[LY-X(ch)]', '[LY-Y(ch)]', '[LY-Z(ch)]', '[MO-X(ch)]', '[MO-Y(ch)]', '[MO-Z(ch)]', '[NE-WX]', '[NE-WY]', '[NE-WZ]', '[LY-WX]', '[LY-WY]', '[LY-WZ]', '[MO-WX]', '[MO-WY]', '[MO-WZ]']


In [41]:
# Create a boolean mask for rows where specified columns have null values
null_mask = pgi_data[cols_with_nulls_lst].isnull()

# Check the shape and content of y_pred
print("Shape of y_pred:", y_pred.shape)
print("Columns in y_pred:", y_pred.columns)

# Apply the mask to update null values with y_pred
for col in cols_with_nulls_lst:
    null_indices = null_mask[col][null_mask[col]].index
    
    # Ensure y_pred has the same indices as null_indices
    y_pred_col = y_pred.loc[null_indices, col].reset_index(drop=True)
    
    # Align the indices of pgi_data for assignment
    pgi_data.loc[null_indices, col] = y_pred_col.values
    
    print(f"Null values after filling {col}:\n{pgi_data[col].isnull().sum()}")

# Verify that the null values have been filled
print("Null values after filling:\n", pgi_data[cols_with_nulls_lst].isnull().sum())


Shape of y_pred: (501, 40)
Columns in y_pred: Index(['NEUT#(10^3/uL)', 'LYMPH#(10^3/uL)', 'MONO#(10^3/uL)', 'EO#(10^3/uL)',
       'BASO#(10^3/uL)', 'NEUT%(%)', 'LYMPH%(%)', 'MONO%(%)', 'EO%(%)',
       'BASO%(%)', 'IG#(10^3/uL)', 'IG%(%)', '[WBC-D(10^3/uL)]',
       '[TNC-D(10^3/uL)]', '[NEUT#&(10^3/uL)]', '[NEUT%&(%)]',
       '[LYMP#&(10^3/uL)]', '[LYMP%&(%)]', '[HFLC#(10^3/uL)]', '[HFLC%(%)]',
       '[BA-D#(10^3/uL)]', '[BA-D%(%)]', '[NE-SSC(ch)]', '[NE-SFL(ch)]',
       '[NE-FSC(ch)]', '[LY-X(ch)]', '[LY-Y(ch)]', '[LY-Z(ch)]', '[MO-X(ch)]',
       '[MO-Y(ch)]', '[MO-Z(ch)]', '[NE-WX]', '[NE-WY]', '[NE-WZ]', '[LY-WX]',
       '[LY-WY]', '[LY-WZ]', '[MO-WX]', '[MO-WY]', '[MO-WZ]'],
      dtype='object')
Null values after filling NEUT#(10^3/uL):
0
Null values after filling LYMPH#(10^3/uL):
0
Null values after filling MONO#(10^3/uL):
0
Null values after filling EO#(10^3/uL):
0
Null values after filling BASO#(10^3/uL):
0
Null values after filling NEUT%(%):
0
Null values after filling 

In [42]:
pgi_data.to_csv('pgi_data_doctor_suggested_Final_Data.csv')